### This Notebook is for Capstone Project for Coursera

# First: Importing libraries

In [146]:
import pandas as pd
import numpy as np
import pgeocode
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# 2nd: importing the data. 

In [147]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df.Borough != 'Not assigned']

# 3rd: getting the coordinates of each Borough

In [149]:
nomi = pgeocode.Nominatim('ca')
df.reset_index(inplace = True)

lat = np.zeros(len(df))
long = np.zeros(len(df))
for k in range(len(df)):
    lat[k] = nomi.query_postal_code(df['Postal Code'][k]).latitude
    long[k] = nomi.query_postal_code(df['Postal Code'][k]).longitude

lat = pd.DataFrame(lat)
long = pd.DataFrame(long)



df['latitude']=lat
df['longitude']=long

df.drop("index", axis=1, inplace=True)

 # 4.Importing json/matplotlib/sklearn libraries 

In [150]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# 5.  Locate under Toronto, CA

In [151]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


# 6. Mapping each borough into the MAP

In [161]:
df.dropna(subset = ["longitude"], inplace=True)
df.reset_index(inplace = True)
df.drop("index", axis=1, inplace=True)

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# 7. Focusing only in boroughs that are named with Toronto

In [247]:
i = 0
toronto_data = pd.DataFrame( columns = df.columns)

for k in range(len(df)):
    if ('Toronto' in df['Borough'][k]):
        toronto_data = toronto_data.append(df.iloc[[k]])

toronto_data.reset_index(inplace = True)
toronto_data.drop("index", axis=1, inplace=True)


# 8. Mapping only these borough

In [163]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Manhattan using latitude and longitude values
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['latitude'], toronto_data['longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

# 9. Preparing to use FourSquare

In [253]:
CLIENT_ID = 'xxxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxxx' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30 # limit of number of venues returned by Foursquare API
radius = 100 # define radius
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: xxxxx
CLIENT_SECRET:xxxxx


# 10. App for getting Venues


In [221]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# 11. Preparing data


In [222]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['latitude'],
                                   longitudes=toronto_data['longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [248]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
col_name="Neighborhood"
first_col = toronto_onehot.pop(col_name)
toronto_onehot.insert(0, col_name, first_col)

#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [249]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0         Cocktail Bar  0.07
1                 Café  0.07
2  Japanese Restaurant  0.07
3       Farmers Market  0.07
4             Beer Bar  0.07


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0               Gift Shop  0.07
1  Thrift / Vintage Store  0.07
2                    Café  0.07
3             Coffee Shop  0.07
4          Breakfast Spot  0.07


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0          Restaurant  0.13
1         Coffee Shop  0.13
2         Yoga Studio  0.07
3  Italian Restaurant  0.07
4         Salad Place  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Italian Restaurant  0.10
1                  Park  0.07
2  Gym / Fitness Center  0.07
3            Restaurant  0.07
4           Coffee Shop  0.07

In [226]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [250]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Café,Farmers Market,Japanese Restaurant,Cocktail Bar,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Breakfast Spot,Gift Shop,Café,Thrift / Vintage Store
2,"Business reply mail Processing Centre, South C...",Restaurant,Coffee Shop,Yoga Studio,Breakfast Spot,Deli / Bodega
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Restaurant,Gym / Fitness Center,Park
4,Central Bay Street,Coffee Shop,Clothing Store,Shopping Mall,Ramen Restaurant,Italian Restaurant
5,Christie,Grocery Store,Café,Athletics & Sports,Playground,Coffee Shop
6,Church and Wellesley,Gay Bar,Juice Bar,Breakfast Spot,Creperie,Burger Joint
7,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Restaurant,Seafood Restaurant,Gym
8,Davisville,Coffee Shop,Dessert Shop,Café,Sandwich Place,Italian Restaurant
9,Davisville North,Gym,Food & Drink Shop,Park,Department Store,Dog Run


In [243]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([8, 8, 1, 1, 1, 5, 1, 8, 8, 5, 5, 8, 3, 1, 6, 4, 8, 8, 7, 1, 5, 0,
       8, 1, 1, 1, 0, 2, 8, 8])

In [251]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighbourhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,1,Coffee Shop,Gym,Bubble Tea Shop,Ramen Restaurant,Portuguese Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,1,Café,Theater,Clothing Store,Shopping Mall,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,8,Café,Farmers Market,Gastropub,Coffee Shop,Gym
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,8,Pub,Trail,Health Food Store,Gastropub,Cheese Shop
5,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,8,Café,Farmers Market,Japanese Restaurant,Cocktail Bar,Beer Bar
6,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860,1,Coffee Shop,Clothing Store,Shopping Mall,Ramen Restaurant,Italian Restaurant
7,M6G,Downtown Toronto,Christie,43.6683,-79.4205,5,Grocery Store,Café,Athletics & Sports,Playground,Coffee Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833,1,Café,Coffee Shop,Restaurant,Seafood Restaurant,Hotel
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378,5,Grocery Store,Furniture / Home Store,Park,Bakery,Gym


# 12. Show data of Toronto's boroughs similarties.


In [252]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We see mainly two clusters, orange and purple. 